# Vector Search for our Semantic Games Recommender System

## Importing Dependencies

In [16]:
from langchain.document_loaders import TextLoader # Takes raw text, convert into format that LangChain can work with
from langchain_text_splitters import CharacterTextSplitter # Split whole document containing descriptions into individual descriptions
from langchain_openai import OpenAIEmbeddings # Make embeddings
from langchain_chroma import Chroma # Store the vectors
from dotenv import load_dotenv # To read in the API keys
import os
import pandas as pd
import warnings
pd.set_option('display.max_colwidth', None)

In [17]:
load_dotenv() # to safely get the API key

True

Since I have saved the OpenAI API key in a `.env` file, using `load_dotenv` will read that file and get the API key safely and securely

## Getting the Data Ready

In [18]:
df = pd.read_csv('cleaned_data.csv')

In [19]:
df.head(2)

,ID,Name,Description,Price,Release Date,Discounted Price,Discount Percentage,Review,Review_2,Links,Tags,Developers,Publishers,Images,Tagged_Description
0,1,Counter-Strike 2,"For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2.",Free,2012-08-21,0,0,Very Positive,"86% of the 8,763,529 user reviews for this game are positive.",https://store.steampowered.com/app/730/CounterStrike_2/?snr=1_7_7_230_150_1,"['FPS', 'Shooter', 'Multiplayer', 'Competitive', 'Action', 'Team-Based', 'eSports', 'Tactical', 'First-Person', 'PvP', 'Online Co-Op', 'Co-op', 'Strategy', 'Military', 'War', 'Difficult', 'Trading', 'Realistic', 'Fast-Paced', 'Moddable']",['Valve'],['Valve'],https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/730/header.jpg?t=1745368595,"1 | For over two decades, Counter-Strike has offered an elite competitive experience, one shaped by millions of players from across the globe. And now the next chapter in the CS story is about to begin. This is Counter-Strike 2."
1,2,Schedule I,"From small-time dope pusher to kingpin - manufacture and distribute a range of drugs throughout the grungy city of Hyland Point. Expand your empire with properties, businesses, employees and more.",£16.75,2025-03-24,0,0,Overwhelmingly Positive,"98% of the 169,380 user reviews for this game are positive.",https://store.steampowered.com/app/3164500/Schedule_I/?snr=1_7_7_230_150_1,"['Simulation', 'Co-op', 'Crime', 'Multiplayer', 'Open World', 'Management', 'First-Person', 'Action', 'Building', 'Crafting', 'Automation', 'Base Building', 'Stealth', 'Strategy', 'FPS', 'Driving', 'Character Customization', 'Shooter', 'Early Access', 'Action-Adventure']",['TVGS'],['TVGS'],https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/3164500/header.jpg?t=1745291685,"2 | From small-time dope pusher to kingpin - manufacture and distribute a range of drugs throughout the grungy city of Hyland Point. Expand your empire with properties, businesses, employees and more."


In [38]:
df['Tagged_Description'].to_csv('tagged_description.txt',
                                 sep='\n',
                                 index= False,
                                 header = False)

In [30]:
raw_documents = TextLoader('tagged_description.txt').load()
enriched = []
for doc in raw_documents:
    
text_splitter = CharacterTextSplitter(chunk_size=0,  # big enough for any game line
                                      chunk_overlap=0,
                                      separator="\n")
documents = text_splitter.split_documents(raw_documents)

Created a chunk of size 228, which is longer than the specified 1
Created a chunk of size 200, which is longer than the specified 1
Created a chunk of size 252, which is longer than the specified 1
Created a chunk of size 132, which is longer than the specified 1
Created a chunk of size 187, which is longer than the specified 1
Created a chunk of size 130, which is longer than the specified 1
Created a chunk of size 285, which is longer than the specified 1
Created a chunk of size 291, which is longer than the specified 1
Created a chunk of size 273, which is longer than the specified 1
Created a chunk of size 220, which is longer than the specified 1
Created a chunk of size 289, which is longer than the specified 1
Created a chunk of size 194, which is longer than the specified 1
Created a chunk of size 267, which is longer than the specified 1
Created a chunk of size 281, which is longer than the specified 1
Created a chunk of size 212, which is longer than the specified 1
Created a 

In [31]:
documents[2]

Document(metadata={'source': 'tagged_description.txt'}, page_content='3 | Lead the members of Expedition 33 on their quest to destroy the Paintress so that she can never paint death again. Explore a world of wonders inspired by Belle Époque France and battle unique enemies in this turn-based RPG with real-time mechanics.')

## Setting up the similarity search/ semantic recommender

In [ ]:
persist_directory = '/Users/mahamedabdulle/Documents/digital futures/capstone/clean_rec' # Change to where you want to save your chroma database
db_games = Chroma.from_documents(
    documents,
    embedding=OpenAIEmbeddings(model="text-embedding-3-large"),
    persist_directory = persist_directory
)


In [41]:
query = "A game that is similar to Expedition 33"
docs = db_games.similarity_search(query,k=10)
docs

[Document(id='11ba3135-826c-471c-998d-b1d1705babc1', metadata={'source': 'tagged_description.txt'}, page_content='1950 | Embark on scientific expeditions with this new off-road adventure game. Lead research missions as you drive a variety of all-terrain vehicles, using high-tech tools and managing your camp and crew to ensure your success in the wild.'),
 Document(id='b210c1b6-eb33-49e5-a776-c79388527eb8', metadata={'source': 'tagged_description.txt'}, page_content='8176 | Curious Expedition is a roguelike expedition simulation set in the late 19th century. Together with famous personalities you will venture on unprecedented expeditions to regions never explored before for fame, science and treasures.'),
 Document(id='3ff58673-9ff7-49cf-b62f-b9741c96ae0d', metadata={'source': 'tagged_description.txt'}, page_content='5084 | eXplore, eXpand, eXploit, eXterminate, EXPRESS. All of the depth, diplomacy, conflict and conquest of a turn-based 4X epic but played in fast-paced 60-minute competi

In [25]:
def retrieve_semantic_recomendations(
    query: str,
    top_k: int = 10,
) -> pd.DataFrame:
    recs = db_games.similarity_search(query,k=50)

    games_list = []

    for i in range(0, len(recs)):
        games_list += [int(recs[i].page_content.strip().replace('"', '').split()[0])]

    return df[df['ID'].isin(games_list)].head(top_k)

In [27]:
retrieve_semantic_recomendations("games that are turn-based")

,ID,Name,Description,Price,Release Date,Discounted Price,Discount Percentage,Review,Review_2,Links,Tags,Developers,Publishers,Images,Tagged_Description
354,355,Total War: WARHAMMER II,"Strategy gaming perfected. A breath-taking campaign of exploration, expansion and conquest across a fantasy world. Turn-based civilisation management and real-time epic strategy battles with thousands of troops and monsters at your command.",£39.99,2017-09-28,0,0,Very Positive,"93% of the 84,149 user reviews for this game are positive.",https://store.steampowered.com/app/594570/Total_War_WARHAMMER_II/?snr=1_7_7_230_150_8,"['Strategy', 'Fantasy', 'Turn-Based Strategy', 'RTS', 'Grand Strategy', 'War', 'Multiplayer', 'Games Workshop', 'Action', 'Turn-Based', 'Singleplayer', 'Dark Fantasy', 'Co-op', 'Tactical', 'Real Time Tactics', 'Story Rich', 'Atmospheric', 'Dwarf', 'Dinosaurs', 'Great Soundtrack']","['CREATIVE ASSEMBLY', 'Feral Interactive (Mac)', 'Feral Interactive (Linux)']","['SEGA', 'Feral Interactive (Mac)', 'Feral Interactive (Linux)']",https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/594570/header.jpg?t=1728485039,"355 | Strategy gaming perfected. A breath-taking campaign of exploration, expansion and conquest across a fantasy world. Turn-based civilisation management and real-time epic strategy battles with thousands of troops and monsters at your command."
429,430,Battle Brothers,"Battle Brothers is a turn based tactical RPG which has you leading a mercenary company in a gritty, low-power, medieval fantasy world. You decide where to go, whom to hire or to fight, what contracts to take and how to train and equip your men in a procedurally generated open world campaign.",£12.49,2017-03-24,£12.49,-50%,Very Positive,"88% of the 21,970 user reviews for this game are positive.",https://store.steampowered.com/app/365360/Battle_Brothers/?snr=1_7_7_230_150_9,"['Strategy', 'RPG', 'Medieval', 'Open World', 'Turn-Based Tactics', 'Turn-Based Strategy', 'War', 'Character Customization', 'Sandbox', 'Tactical', 'Simulation', 'Action', 'Tactical RPG', 'Replay Value', 'Turn-Based Combat', 'Moddable', 'Turn-Based', 'Singleplayer', 'Historical', 'Adventure']",['Overhype Studios'],['Overhype Studios'],https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/365360/header.jpg?t=1746327392,"430 | Battle Brothers is a turn based tactical RPG which has you leading a mercenary company in a gritty, low-power, medieval fantasy world. You decide where to go, whom to hire or to fight, what contracts to take and how to train and equip your men in a procedurally generated open world campaign."
508,509,Total War: WARHAMMER,"Addictive turn-based empire-building with colossal, real-time battles, all set in a world of legendary heroes, giant monsters, flying creatures and storms of magical power.",£39.99,2016-05-24,0,0,Mostly Positive,"78% of the 31,595 user reviews for this game are positive.",https://store.steampowered.com/app/364360/Total_War_WARHAMMER/?snr=1_7_7_230_150_11,"['Strategy', 'Fantasy', 'RTS', 'War', 'Turn-Based Strategy', 'Games Workshop', 'Grand Strategy', 'Multiplayer', 'Tactical', 'Dark Fantasy', 'Action', 'Singleplayer', 'Turn-Based', 'Dwarf', 'Atmospheric', 'Co-op', 'Story Rich', 'Open World', 'RPG', 'Gore']","['CREATIVE ASSEMBLY', 'Feral Interactive (Linux)', 'Feral Interactive (Mac)']","['SEGA', 'Feral Interactive (Linux)', 'Feral Interactive (Mac)']",https://shared.fastly.steamstatic.com/store_item_assets/steam/apps/364360/header.jpg?t=1668768531,"509 | Addictive turn-based empire-building with colossal, real-time battles, all set in a world of legendary heroes, giant monsters, flying creatures and storms of magical power."
767,768,Songs of Conquest,"Raise mighty armies, wield ancient magic and forge an empire. This turn-based strategy adventure game fuses strategic decision making, tactical combat and kingdom management.",£10.03,2024-05-20,£10.03,-66%,Very Positive,"86% of the 10,294 user reviews for this game are posit